# Homework 7 - Camilo Gómez Zapata
### C.C.1010088965

Teach a NN model to predict the prices of the houses from the `kc_house_data.csv'` dataset. 

* clean the data (NaN, repeated values, correlation...)
***I forgot to mention that the target (here `price`) should not be taken into account when making the correlation matrix***!!
* remove outliers (very large, very small values)
* split into train and test 
* use functions:
  * `get_model(input_shape)`
  * `get_regularized_model(input_shape,weight_decay,drop_ate)`
  * `compile_model()`
* use `metrics=['mape']` when compiling model
* `scores=model.evaluate(X_test,y_test)` should return mean absolute percentage error
* use ModelCheckpoint() callback to save the entire model into `.h5` file
* (you can possibly normalize the data before training the algorithm) check this out

    * "id": Numero de identificacion de la vivienda
    * "Date": Fecha en que se vendió la casa
    * "Price": el precio es el objetivo de predicción
    * "Bedrooms": Número de Dormitorios/Casa
    * "Bathrooms": Número de baños/Casa
    * "Sqft_Living": pies cuadrados de la casa
    * "Sqft_Lot": pies cuadrados del lote
    * "Floors": Total de pisos oniveles) en casa
    * "Waterfront": Casa que tiene vista a la costanera
    * "View": Ha sido visto
    * "Condition": Qué tan buena es la condición ogeneral)
    * "Grade": calificación de vivienda, según el sistema del condado de King
    * "Sqft_above": pies cuadrados de la casa aparte del sótano
    * "Sqft_Basement": pies cuadrados del sótano
    * "Yr_Built": año de construcción
    * "Yr_Renovated": año en que se renovó la casa
    * "Zipcode": código postal
    * "Lat": Coordenada de latitud
    * "Long": Coordenada de longitud
    * "Sqft_Living15": Área de la sala de estar en 2015 oimplica algunas renovaciones)
    * "Sqft_Lot15": área del tamaño del lote en 2015 oimplica algunas renovaciones) 

1. Importar, leer e inspeccionar los datos
  * `df.head()`
  * `df.tail()` 
  * `df.columns()`
  * `df.describe()`
  * `df.info()`
  * `df.loc[mascara]` - selecione filas usando "mascara" (ejemplo de mascara (condicion logica): `df['id']>100`) 
2. Completar los datos y organizar las columnas
  * `df.merge()`
  * `df[df.duplicated()] ` - visualiza filas con valores duplicadas atraves todas columnas
  * `df.drop_duplicates(subset=['nombre_columna'])` - eliminar valor duplicadas en la columna
  * `df.drop(['nombre_columna'], axis=1)`           - eliminar columnas innecesarias o repetidas usando funcion
  * `df['nueva_columna']=df['nueva_columna']/10.764` añadir otros nuevos (utilizando cálculos)
  * `df.rename(columns={'nombre_columna_vieja': 'nombre_columna_nueva'})` cambiar el nombre de las columna
  * `df['nombre_columna'] = df['nombre_columna'].astype(str)` cambiar tipo de valores de la columna
3. Comprobar de los tipos de datos y eliminar de los valores repetidos, y `Nan`
  * `df.info()` 
  * `df.isnull().sum()` - soma de todas valores vacios en cada columna
  * `df.isna().sum()`  - soma de todas valores NaN en cada columna
  * `df['sqft_lot'].isnull()` - mascara logica de todos valores null util combinando con `df[mascara]`
  * `df.dropna()` - elimina valores Nan, parametro `axis=0` significa eliminar todas filas con valor NaN
4. Revisar y comprender los datos
  * `df['nombre_columna'].unique()`      - visualizar valores uniqas de la columna
  * `df['nombre_columna'].value_counts()` - sumar frequencia de valores de la columna (se usa para columnas con valores discretos)
  * `df.groupby(['nombre_columna_1','nombre_columna_1']).mean()`
  * `df['nombre_columna'].nlargest(10)` - visualiza 10 mas grandes valore numericas de la columna
  * `df.info()`
  * `df.describe()`
5. Compruebar la correlación entre variables y eliminar las que tengan una correlación superior a 0,8 
  * `corr()`
6. Buscar y eliminar valores atípicos
  * mirar las estadísticas de `df.describe()` max() min()   
  * `df.groupby`
  * histogramas 
  * IQR y boxplot `seaborn.boxplot()`
7. Comprobar la normalidad de la distribución de continua variable
  * `distplot()`
8. Búscar de linealidad y relación general entre variables
  * `pairplot()`
9. Identificar de variables categóricas
  * `matplotlib.scatter_plot()`
  * `seaborn.scatter_plot()`
10. Extraer otras informaciónes válida, por ejemplo usando series de tiempo (time series )
  * `pandas.to_datetime()`
  * `pandas.DatetimeIndex()`

## Solution

#### 1. Clean the data (NaN, repeated values, correlation...)

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('kc_house_data.csv', sep=',')

Drop NAs, drop duplicate rows, drop rows with duplicate ids, drop id and date column, turn floors column into float, turn zipcode column into int.

In [2]:
data = data.dropna()
data = data.drop_duplicates()
data = data.drop_duplicates("id")
data = data.drop(["id", "date", "waterfront", "yr_renovated"], axis=1)
data["floors"] = data["floors"].astype(float)
data["zipcode"] = data["zipcode"].astype(int)

Drop highly correlated values (above 0.8).

In [3]:
data.corr()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15
price,1.000000,0.308964,0.525445,0.701981,0.088883,0.256248,0.396929,0.035131,0.667525,0.605484,0.323533,0.052990,-0.052078,0.306017,0.020538,0.584668,0.082027
bedrooms,0.308964,1.000000,0.516768,0.577490,0.031699,0.175156,0.079525,0.027961,0.358435,0.478424,0.303008,0.153786,-0.153493,-0.010318,0.129792,0.393119,0.029099
bathrooms,0.525445,0.516768,1.000000,0.754420,0.087137,0.499855,0.186679,-0.127822,0.664739,0.684990,0.283528,0.505108,-0.202933,0.023416,0.221922,0.568189,0.086621
sqft_living,0.701981,0.577490,0.754420,1.000000,0.171948,0.352841,0.283746,-0.060937,0.762431,0.876681,0.434404,0.316572,-0.198552,0.051066,0.239190,0.756195,0.182767
sqft_lot,0.088883,0.031699,0.087137,0.171948,1.000000,-0.005963,0.075008,-0.009285,0.112546,0.182574,0.015098,0.052454,-0.129395,-0.086013,0.229548,0.143639,0.718096
floors,0.256248,0.175156,0.499855,0.352841,-0.005963,1.000000,0.028198,-0.266514,0.457498,0.522636,-0.246101,0.489094,-0.058020,0.049227,0.124256,0.278822,-0.011930
view,0.396929,0.079525,0.186679,0.283746,0.075008,0.028198,1.000000,0.045065,0.249895,0.166923,0.276476,-0.055146,0.087139,0.005653,-0.079732,0.279070,0.072863
condition,0.035131,0.027961,-0.127822,-0.060937,-0.009285,-0.266514,0.045065,1.000000,-0.148263,-0.160758,0.174496,-0.364976,0.004415,-0.015348,-0.107751,-0.094962,-0.003719
grade,0.667525,0.358435,0.664739,0.762431,0.112546,0.457498,0.249895,-0.148263,1.000000,0.756086,0.167043,0.445325,-0.182727,0.113354,0.196586,0.712541,0.118318
sqft_above,0.605484,0.478424,0.684990,0.876681,0.182574,0.522636,0.166923,-0.160758,0.756086,1.000000,-0.052477,0.422860,-0.260384,-0.001692,0.343099,0.731985,0.193407


In [4]:
data = data.drop(["sqft_above", "sqft_living"], axis=1)

#### 2. Remove outliers (very large, very small values)

Remove all rows that have at least one column with a value that is more than 3 standard deviations away from the mean of that column.

In [5]:
from scipy import stats

data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]

#### 3. Scale and split into train and test

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = data.drop(["price"], axis=1).to_numpy()
y = data["price"].to_numpy().reshape((len(data["price"].to_numpy()), 1))

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_X.fit(X)
scaler_y.fit(y)
X_scaled = scaler_X.transform(X)
y_scaled = scaler_y.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1)

#### 4. Create model

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Softmax, InputLayer

def get_model(input_shape):
    return Sequential([
        InputLayer(input_shape=input_shape),
        Dense(units=16, activation='relu'),
        Dense(units=16, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])

def compile_model(model):
    model.compile(
        optimizer='adam',
        loss='MSE',
        metrics=['mape']
    )


model = get_model((X.shape[1],))
compile_model(model)

#### 5. Train with checkpoints

In [8]:
checkpoint_filepath = "C:/Users/Camilo/Documents/Estudio/Redes Neuronales/NeuralNetworkRSouzaCourse/Notebooks/extras/HomeworksSubmissions/1010088965/Homework_07/tmp/checkpoint"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model.h5',
    save_weights_only=False,
    frequency='epoch',
    verbose=1
)

model.fit(X_train, y_train, epochs=32, batch_size=32, verbose=2, callbacks=[checkpoint])

Epoch 1/32

Epoch 00001: saving model to model.h5
544/544 - 0s - loss: 0.0171 - mape: 14914.0449
Epoch 2/32

Epoch 00002: saving model to model.h5
544/544 - 0s - loss: 0.0074 - mape: 6134.1255
Epoch 3/32

Epoch 00003: saving model to model.h5
544/544 - 0s - loss: 0.0067 - mape: 5311.9399
Epoch 4/32

Epoch 00004: saving model to model.h5
544/544 - 0s - loss: 0.0064 - mape: 5288.2095
Epoch 5/32

Epoch 00005: saving model to model.h5
544/544 - 0s - loss: 0.0062 - mape: 5385.4429
Epoch 6/32

Epoch 00006: saving model to model.h5
544/544 - 0s - loss: 0.0061 - mape: 5145.8047
Epoch 7/32

Epoch 00007: saving model to model.h5
544/544 - 0s - loss: 0.0059 - mape: 5188.0859
Epoch 8/32

Epoch 00008: saving model to model.h5
544/544 - 0s - loss: 0.0057 - mape: 5462.1431
Epoch 9/32

Epoch 00009: saving model to model.h5
544/544 - 0s - loss: 0.0056 - mape: 5525.9517
Epoch 10/32

Epoch 00010: saving model to model.h5
544/544 - 0s - loss: 0.0055 - mape: 5415.9185
Epoch 11/32

Epoch 00011: saving model

#### 6. Evaluate the model

In [9]:
scores = model.evaluate(X_test, y_test)
scores

61/61 [==============================] - 0s 492us/step - loss: 0.0046 - mape: 21.8503


[0.0046317242085933685, 21.85031509399414]

As you can see, the MAPE is of 21%, which means that the model is providing decent results.